In [ ]:
!git clone https://github.com/HuangRihChang/GSHSD.git
%cd GSHSD/GSHSD
!ls

In [ ]:
from IPython.display import clear_output
import time

!rm ./ngrok 
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
clear_output()
!./ngrok authtoken 23DuHXStdM98jNNlyZIVaHNFktk_8agkCvS2XmfuiY1yR5izh

In [ ]:
! rm -r ./runs/
! mkdir "./runs/"

LOG_DIR = './runs/'
get_ipython().system_raw(f'tensorboard --logdir {LOG_DIR} --host 0.0.0.0 --port 2111 &')
get_ipython().system_raw('./ngrok http 2111 &')
time.sleep(10)
clear_output()
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
time.sleep(10)

In [ ]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

clear_output()
# !pip install tensorboard

In [ ]:
from vncorenlp import VnCoreNLP

class Segmentation:
    def __init__(self, api="./vncorenlp/VnCoreNLP-1.1.1.jar", ):
        self.rdrsegmenter = VnCoreNLP(api, annotators="wseg", max_heap_size='-Xmx500m')

    def __call__(self, text):
        res = ""
        sentences = self.rdrsegmenter.tokenize(text.lower())
        for sentence in sentences:
            res += (" ".join(sentence)) + " "
        return res.strip()

In [ ]:
import functools

import torch
from torch import nn
from model.model import TransformerEncoder
from model.metrices import F1Score
from model.losses.softmax import CenterLoss, FocalLoss
from HSDDataset import ViHSDData

from torch.utils.data import DataLoader, RandomSampler
from tqdm.notebook import tqdm
# from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
from utils import get_device
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import emoji
import re
import math

In [ ]:
torch.manual_seed(8596)

LR = 1e-6
n_epochs = 15
classes_num = 3
accum_iter = 2
batch_size = 64
checkpoint_batch_size = 1024
max_len = 128
device = get_device()

writer = SummaryWriter()

In [ ]:
segmentator = Segmentation()

def proprocess(x):
    x = str(x)
    x = emoji.replace_emoji(x, replace='')
    x = segmentator(x)
    x = re.sub(r" +", " ", x)
    return x.lower().strip()

In [ ]:
# Defining Model for specific fold "vinai/phobert-base"
# model_path = "../../../input/mbert-model/bert-base-multilingual-uncased"
model_path = "../../../input/phobertpretrained/"
# model_path = "./weights/multiBERTuncased"

model = TransformerEncoder(model_path, classes_num = classes_num,
                            max_seq_length = max_len, 
                            checkpoint_batch_size = checkpoint_batch_size,
                            dropout_rate = 0.1, 
                            residial = True,
                            model_args = {"output_hidden_states":False}
                            )

In [ ]:
import random

unk_token = model.tokenizer.unk_token

def get_item(row, mask_rate=0.1):
    utterance, hate_label = row["free_text"], row["label_id"]
    utterance = utterance.split()
    mask_rate = random.uniform(0., mask_rate)
    start = random.randint(0,len(utterance))
    end = min(random.randint(start,len(utterance)), start+int(len(utterance)*mask_rate))
    tmp = utterance[0:start] + [unk_token]*(end-start) + utterance[end:]
    utterance = " ".join(tmp)
    return utterance, hate_label

In [ ]:
model_collate_fn = functools.partial(lambda x: x)


train_df = pd.read_csv("./data/vihsd/train.csv")
# max_ = 0
# for i in range(len(train_df.label_id.unique())):
#     tmp = train_df[train_df["label_id"]==i]
#     if len(tmp) >= max_:
#         max_ = len(tmp)

# for i in range(len(train_df.label_id.unique())):
#     tmp = train_df[train_df["label_id"]==i]
#     for _ in range(math.ceil(max_/len(tmp))-1):
#         train_df = pd.concat([train_df, tmp])

train_data = ViHSDData(train_df, 
                 utterance_feild = "free_text", 
                 label_feild="label_id", 
                 text_preprocessor=proprocess,
                 augment_fnct=get_item,
                )
train_sampler = RandomSampler(train_data)
data_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, collate_fn=model_collate_fn)

val_df = pd.read_csv("./data/vihsd/dev.csv")
val_data = ViHSDData(val_df, 
                 utterance_feild = "free_text", 
                 label_feild="label_id", 
                 text_preprocessor=proprocess
                )
val_sampler = RandomSampler(val_data)
val_loader = DataLoader(val_data, batch_size=batch_size, sampler=val_sampler, collate_fn=model_collate_fn)


test_df = pd.read_csv("./data/vihsd/test.csv")
test_data = ViHSDData(test_df, 
                 utterance_feild = "free_text", 
                 label_feild="label_id", 
                 text_preprocessor=proprocess
                )
test_sampler = RandomSampler(test_data)
test_loader = DataLoader(test_data, batch_size=batch_size, sampler=test_sampler, collate_fn=model_collate_fn)

training_step = len(data_loader)*n_epochs
print(f"Total {training_step} training steps for this dataset")

In [ ]:
labels = list(train_df.label_id.unique())
for i in labels:
    tmp = train_df[train_df.label_id == i]
    print(len(tmp), 1 - len(tmp)/len(train_df))

In [ ]:
# cross_entropy_loss = nn.CrossEntropyLoss()
classes_weights = torch.Tensor([0.1730705256154358, 0.8937125748502994, 0.9332168995342648]).to(device)
cross_entropy_loss = FocalLoss(alpha = classes_weights, gamma=1.2)

center_loss = CenterLoss(num_class=classes_num,\
                         num_feature = model.get_word_embedding_dimension()
                        )

model.to(device)
cross_entropy_loss.to(device)
center_loss.to(device)

model_params = list(model.named_parameters()) # included all params from pooler and transformers
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
model_params = [{'params': [p for n, p in model_params if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0001},
                {'params': [p for n, p in model_params if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
               ]

opt_params = model_params

model_optimizer = AdamW(opt_params, lr=LR)
model_scheduler = get_linear_schedule_with_warmup(model_optimizer,
                                            num_warmup_steps=training_step*0.5, 
                                            num_training_steps=training_step
                                            )

center_loss_params = list(center_loss.named_parameters())
center_loss_params = [{'params': [p for n, p in center_loss_params], 'weight_decay': 0.0},]
center_loss_optimizer = AdamW(center_loss_params, lr=0.1)
center_loss_scheduler = get_linear_schedule_with_warmup(center_loss_optimizer,
                                            num_warmup_steps=training_step*0.5, 
                                            num_training_steps=training_step
                                            )
# for param in model.sent_encoder.embeddings.parameters():
#     param.requires_grad = False

In [ ]:
def make_batch(batch, tokenizer, max_len="dynamic", device="cuda:0"):
    text_list, labels = [text for text,_ in batch], [label for _,label in batch]
    
    if max_len == "dynamic":
        lengths = np.array([len(tokenizer.tokenize(x))+2 for x in text_list])
        max_len = int(lengths.max())
    
    labels = torch.LongTensor(labels).to(device)
    toks = tokenizer.batch_encode_plus(text_list, max_length=max_len, padding='max_length', truncation=True)
    ids, mask = (torch.LongTensor(toks["input_ids"]).to(device), torch.LongTensor(toks["attention_mask"]).to(device))
    inputs = {"input_ids": ids, "attention_mask": mask}
    return inputs, labels

In [ ]:
def train(data_loader, model, cross_entropy_loss, center_loss, train_step):
    tk = tqdm(data_loader)
    f1 = F1Score()
    model.train()
    for i, x in enumerate(tk):
        inputs, labels = make_batch(x, model.tokenizer, max_len=max_len, device=device)
        if train_step == 0:
            writer.add_graph(model, inputs)
        
        with torch.set_grad_enabled(True):
            logits, features = model(inputs)
            celoss = cross_entropy_loss(logits, labels)
            loss = celoss + 0.001*center_loss(features, labels)
            loss.backward()
        
        
        if ((i + 1) % accum_iter == 0) or (i + 1 == len(data_loader)):
            for param in center_loss.parameters():
                if param.grad is not None:
                    param.grad *= 1/0.0001

            model_optimizer.step()
            center_loss_optimizer.step()
            model_scheduler.step()
            center_loss_scheduler.step()
            model_optimizer.zero_grad()
            center_loss_optimizer.zero_grad()
        
        with torch.no_grad():
            predict = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
            macro_f1 = f1(predict+1, labels+1, "macro")
            tk.set_postfix(Epoch=e, step=train_step, loss=loss.data.item(), f1=macro_f1[0].data.item())
            writer.add_scalar('loss', loss.data.item(), train_step)
            writer.add_scalar('celoss', celoss.data.item(), train_step)
            writer.add_scalar('macro_f1', macro_f1[0].data.item(), train_step)
        train_step += 1
    return train_step


def evaluation(dev_loader, model, cross_entropy_loss, prefix=None):
    f1 = F1Score()
    logits_list, labels_list = [], []
    model.eval()
    with torch.no_grad():
        for x in dev_loader:
            inputs, labels = make_batch(x, model.tokenizer, max_len=max_len, device=device)
            logits, _ = model(inputs)
            logits_list.append(logits)
            labels_list.append(labels)
        labels = torch.cat(labels_list, dim=0)
        logits = torch.cat(logits_list, dim=0)
        predicts = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
        macro_f1 = f1(predicts+1, labels+1, "macro")
        loss = cross_entropy_loss(logits, labels)
        print(f"Epoch {e} - loss: {loss.data.item()} - F1: {macro_f1[0].data.item()}")
        if prefix is not None:
            writer.add_scalar(f'{prefix} celoss', loss.data.item(), e)
            writer.add_scalar(f'{prefix} macro_f1', macro_f1[0].data.item(), e)
            
        print(confusion_matrix(labels.cpu().detach().numpy(), 
                               torch.argmax(torch.softmax(logits, dim=-1), dim=-1).cpu().detach().numpy()))
    return logits, labels, loss, macro_f1

In [ ]:
! rm -r ./runs/
! mkdir "./runs/"
writer = SummaryWriter()

train_step = 0
best_f1 = 0
best_loss = 1e8
for e in range(n_epochs):
    train_step = train(data_loader, model, cross_entropy_loss, center_loss, train_step)
    _, _, loss, macro_f1 = evaluation(val_loader, model, cross_entropy_loss,  prefix="val")
    
    val_loss = loss.data.item()
    val_f1_score = macro_f1[0].data.item()
    
    if best_f1 <= val_f1_score:
        best_f1 = val_f1_score
        print(f"Epcoh {e} Saving best f1_model with score {best_f1}")
        torch.save(model.state_dict(), f"best_f1.pth")
    
    if best_loss >= val_loss:
        best_loss = val_loss
        print(f"Epcoh {e} Saving best loss model with score {best_loss}")
        torch.save(model.state_dict(), f"best_loss.pth")

logits, labels, loss, macro_f1 = evaluation(test_loader, model, cross_entropy_loss)

In [ ]:
model.load_state_dict(torch.load("best_f1.pth"))
model.eval()
e = 11
logits, labels, loss, macro_f1 = evaluation(test_loader, model, cross_entropy_loss)
predicts = torch.argmax(torch.softmax(logits, dim=-1), dim=-1).cpu().detach().numpy()
labels = labels.cpu().detach().numpy()
loss, macro_f1, confusion_matrix(labels, predicts)

In [ ]:
model.load_state_dict(torch.load("best_loss.pth"))
model.eval()
e = 11
logits, labels, loss, macro_f1 = evaluation(test_loader, model, cross_entropy_loss)
predicts = torch.argmax(torch.softmax(logits, dim=-1), dim=-1).cpu().detach().numpy()
labels = labels.cpu().detach().numpy()
loss, macro_f1, confusion_matrix(labels, predicts)